터미널에서 해당 명령어 실행해서 크롬 원격 디버그창을 띄워놓은 상태에서 코드 실행

[windows]

start chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\temp"

[MacOS]

sudo /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --remote-debugging-port=9222

In [18]:
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.chrome.options import Options
import time

이성희님

1. 기능을 함수로 쪼개서 함수별로 주석이 있으면 좋겠습니다

2. while 문 진입 후 finde_element 가 없을경우 exception이 던져지기 때문에 try-catch가 있으면 좋을것 같아요

In [ ]:
options = Options()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver2 = webdriver.Chrome(options=options)

driver.get("https://slack.com/signin#/signin")

driver2.get("https://chat.openai.com/")
driver2.implicitly_wait(10)


id_input = driver.find_element(By.ID, "signup_email")
actions = ActionChains(driver)
actions.send_keys_to_element(id_input, "rudah69@gmail.com").perform()
actions.send_keys(Keys.RETURN).perform()

input("Press Enter to continue...") 
## Chat GPT의 경우에는 구글 로그인같은 내역을 남겨놔야 접속이 되므로 완전 자동화는 힘들다.
## 여기서 로그인 -> 채널 접속까지 한 뒤에 엔터키를 누른다. 
slack_recent_message = ""
slack_to_gpt_input = ""
recent_answer = ""

while True:
    chats = driver.find_elements(By.CLASS_NAME, "p-rich_text_section")

    if chats[-1].text == slack_recent_message:
        # 최근 메세지가 이전에 받았던 메세지와 동일할 경우 스킵
        print("슬랙 메세지가 업데이트 되지 않았습니다.")
        time.sleep(5)
        continue
    elif chats[-1].text in recent_answer:
        # 자문 자답 방지용, 대화 내용이 최근 답변과 유사할 경우 스킵
        print("슬랙 메세지가 업데이트 되지 않았습니다.")
        time.sleep(5)
        continue
        
    slack_recent_message = chats[-1].text

    gpt_input = driver2.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[2]/form/div/div[1]/textarea')
    gpt_button = driver2.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[2]/form/div/div[1]/button')

    actions_gpt = ActionChains(driver2)
    actions_gpt.send_keys_to_element(gpt_input, slack_recent_message).perform()
    actions_gpt.click(gpt_button).perform()
    
    driver2.implicitly_wait(3)
    main_divs = driver2.find_elements(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[1]/div/div/div/div')
    answer_num = len(main_divs) - 1
    
    delay_check = 0 #Chat GPT가 대답하는 과정이 길기 때문에 대답 하는 동안에는 체크하다가 대답이 끝나면 전송

    recent_answer = driver2.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[1]/div/div/div/div[%s]/div/div[2]/div[1]/div/div' % answer_num).text
    while delay_check <= 2:
        time.sleep(0.5)
        tmp_answer = driver2.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/main/div[1]/div/div/div/div[%s]/div/div[2]/div[1]/div/div' % answer_num).text
        if recent_answer == tmp_answer:
            delay_check += 1
        else:
            delay_check = 0
            recent_answer = tmp_answer
    
    slack_input = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[1]/div[5]/div[2]/div/div[4]/div/div[2]/div/div/div/div[2]/div/div/div/div[2]/div/div[1]/p')
    actions.send_keys_to_element(slack_input, recent_answer).perform()
    actions.send_keys(Keys.RETURN).perform()